<a href="https://colab.research.google.com/github/Summarizer-cb/Gas_Energy_Summarization/blob/main/%5B%EC%B0%BD%EC%9A%B0%5D1208_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
import os
import math
import json
import time
import random
import multiprocessing
import json
import glob
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

import numpy as np
import pandas as pd
from tqdm.auto import tqdm, trange
from sklearn import model_selection

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Parameter
import torch.optim as optim
from torch.utils.data.distributed import DistributedSampler
from torch.utils.data import (
    Dataset, DataLoader,
    SequentialSampler, RandomSampler
)
import transformers
from transformers import (AutoModel, AutoTokenizer, AdamW, BartForConditionalGeneration)


In [ ]:
with open('/<your_dir>/new_test.jsonl', "r", encoding="utf-8") as f:
    jsonl = list(f)
test_data = []
for json_str in jsonl:
    test_data.append(json.loads(json_str))
len(test_data)


6596

In [ ]:
dict_test = {'id':[],'content':[],'media':[]}
for data in test_data:
  dict_test['id'].append(data['id'])
  dict_test['content'].append(data['article_original'])
  dict_test['media'].append(data['media'])
test_set = pd.DataFrame(dict_test)
test_set

,id,content,media
0,472779091,[일본 사법당국의 출국금지 명령이 내려져 있던 카를로스 곤(65) 전 르노닛산 회장...,중앙일보
1,472779092,"[본선에서 트럼프에 패할 수 있다는 우려 때문이다., 지지율 3위: 엘리자베스 워런...",중앙일보
2,472779093,"[1월 1일 새해 김정은 북한 국무위원장은 모습을 드러내지 않았다., 2012년 집...",중앙일보
3,472779094,[국내 이동통신 3사(KT·SK텔레콤·LG유플러스)는 산토끼(신규 고객)를 잡느라 ...,중앙일보
4,472779095,[카를로스 곤(66)닛산자동차 전 회장의 ‘보석기간 중 탈출극’으로 인한 일본내 충...,중앙일보
...,...,...,...
6591,472785682,"[제1조(목적)"", ""이 고시는 「온실가스 배출권의 할당 및 거래에 관한 법률」(이...",환경부
6592,472785683,[산업통상자원부와 한국가스안전공사는 겨울철 불법 가스용품 유통에 따른 가스사고 예방...,헤럴드경제
6593,472785684,[정부가 올해 안에 2050 탄소중립 시나리오와 2030년 국가 온실가스감축목표(N...,머니투데이
6594,472785685,[정부가 수소차 보급 활성화를 위해 수소 제조용 천연가스 요금을 한시적으로 낮춘다....,경향신문


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("hyunwoongko/kobart")
added_token_num = tokenizer.add_special_tokens({"additional_special_tokens":["<GSG>"]})
tokenizer.special_tokens_map

{'additional_special_tokens': ['<GSG>'],
 'bos_token': '<s>',
 'cls_token': '<s>',
 'eos_token': '</s>',
 'mask_token': '<mask>',
 'pad_token': '<pad>',
 'sep_token': '</s>',
 'unk_token': '<unk>'}

In [ ]:
class TestDataset(torch.utils.data.Dataset):
  def __init__(self,df):
    self.tokenizer = tokenizer
    self.df = df
  
  def __getitem__(self,idx):
    doc = self.df.iloc[idx]
    input = ' '.join(doc['content'])
    id = doc['id']

    input_text = tokenizer.batch_encode_plus([input],padding='max_length',truncation=True, max_length=512, return_tensors="pt")

    return {"id":id,
            "input_ids": input_text['input_ids'].flatten(), 
            "attention_mask": input_text['attention_mask'].flatten()
            }

  def __len__(self):
      return self.df.__len__()

  def __repr__(self):
    return f"data size : {self.__len__()} "

testdataset = TestDataset(test_set)

In [ ]:
batch_size = 6
test_dataloader = DataLoader(testdataset, shuffle=False, batch_size=batch_size)
for sample in test_dataloader:
  print(sample)
  break

{'id': ['472779091', '472779092', '472779093', '472779094', '472779095', '472779096'], 'input_ids': tensor([[18007, 19014,  9770,  ...,     3,     3,     3],
        [10888, 21141, 18751,  ...,     3,     3,     3],
        [  248, 11940, 16476,  ...,     3,     3,     3],
        [23436, 22915, 14086,  ...,     3,     3,     3],
        [29597, 24080, 17297,  ...,     3,     3,     3],
        [10746, 16420, 23383,  ...,     3,     3,     3]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}


## Load Weights

In [ ]:

pretrained_weight = "/<your_dir>/KoPEGASUS_Finetuning_1_done_0.00000.pt"
pretrained_weight

'/content/drive/MyDrive/가스에너지문서요약/Weights/KoPEGASUS_Finetuning_1_done_0.00000.pt'

In [ ]:
model = BartForConditionalGeneration.from_pretrained("hyunwoongko/kobart",return_dict=True)
model.resize_token_embeddings(tokenizer.vocab_size + added_token_num)
model.load_state_dict(torch.load(pretrained_weight))  # GPU 사용시

<All keys matched successfully>

In [ ]:
# 기사 원문
for i in range(4):
  print(' '.join(test_set.iloc[i]['content']))

일본 사법당국의 출국금지 명령이 내려져 있던 카를로스 곤(65) 전 르노닛산 회장이 레바논으로 비밀리에 도주했다. 30일(현지시간) 미 일간 월스트리트저널(WSJ)은 소식통을 인용해 곤 전 회장이 이날 레바논 수도 베이루트 공항에 도착했다고 전했다. 곤 전 회장은 보도 이후 미국의 대변인을 통해 자신이 레바논에 머물고 있다는 내용의 성명을 발표했다. 이 때문에 곤 전 회장의 변호인은 물론 수사·출입국 당국까지 당혹스럽다는 반응을 보이는 상황이다. 곤 전 회장은 일본의 ‘정치적 박해’로부터 빠져나왔다고 주장하고 있다. 그는 성명을 통해 “유죄가 전제되고 차별이 만연하고 기본적 인권이 무시되는 잘못된 일본 사법제도의 ‘인질’이 되지 않겠다”고 밝혔다. 곤 전 회장은 르노닛산 회장 시절 회사 공금을 유용했다는 혐의 등으로 일본에서 재판을 받고 있다. 지난해 4월 두 번째 보석 당시 법원에서는 거주제한·출국금지 조치를 포함한 엄격한 감시 명령을 내렸다. 도쿄지방재판소는 도쿄도(東京都) 미나토(港)구 소재 단독 주택으로 곤의 주거지를 제한하고 일본에서 출국하는 것을 금지하는 등의 조건으로 보석을 인정했다. 곤 전 회장의 주거지 현관에 감시 카메라를 설치해 녹화된 내용을 정기적으로 법원에 제출하며, 휴대전화의 경우 인터넷 접속을 할 수 없는 1대만 변호인에게서 받아 사용하고 그 통화 이력도 법원에 제출하도록 하는 조건도 걸려 있었다. 곤 전 회장의 변호인 히로나카 준이치로(弘中惇一郞) 변호사는 “보도된 내용 이상의 것을 알지 못하며 현재 상황에 매우 놀랐다. 앞으로 정보가 들어오면 법원에 제공하겠다”고 반응했다. 그는 “곤 전 회장의 여권은 변호사가 보관하고 있으며 변호인단이 여권을 주는 일은 있을 수 없다”고 설명했다. 교도통신은 일본 출입국재류관리청의 데이터베이스에 곤 전 회장이 출국한 기록이 없는 것으로 확인됐다고 관계자의 설명을 토대로 전했다. 이와 관련해 레바논 치안 당국자는 곤 전 회장으로 보이는 인물이 개인용 제트기를 이용해 베이루트에 도착했다고 NHK에 설

In [ ]:
# forwarding 결과
output = model.forward(sample['input_ids'])
tokenizer.batch_decode(output.logits.argmax(2))

['일본 사법당국의 출국금지 명령이 내려져 있던 카를로스 곤(65) 전 르노닛산 회장이 레바논으로 비밀리에 도주한</s>일(현지시간) 미 일간 월스트리트저널(WSJ)은 소식통을 인용해 곤 전 회장이 이날 레바논 수도 베이루트 공항에 도착했다고 전하며</s> 전 회장은 보도 이후 미국의 대변인을 통해 자신이 레바논에 머물고 있다는 내용의 성명을 발표했다.</s> 때문에 곤 전 회장의 변호인은 물론 수사·출입국 당국까지 당혹스럽다는 반응을 보이고 상황이다.</s> 전 회장은 일본의 ‘정치적 박해’로부터 빠져나왔다고 주장하고 있다.</s> “ 통해 유유죄가 전제되고 차별이 만연하고 기본적 인권이 무시되는 잘못된 일본 사법제도의 인인질’이 되지 않겠다”고 밝혔다.</s> 전 회장이 일본닛산 회장 시절 회사 공금을 유용했다는 혐의 등으로 일본에서 재판을 받고 있다.</s> 4월 두 번째 보석 당시 법원에서는 거주제한·출국금지 조치를 포함한 엄격한 감시 명령을 내렸다.</s>지방재판소는 도쿄도 미東京都) 미나토(港)구 소재 단독 주택으로 곤의 주거지를 제한하고 일본에서 출국하는 것을 금지하는 조건으로 조건으로 보석을 인정했다.</s> 전 회장이 여지를 현관에 감시 카메라를 설치해 녹화된 내용을 정기적으로 법원에 제출하며 휴대전화의 경우 인터넷 접속을 할 수 없는 1대만 변호인에게서 받아 사용하고 그 통화 이력도 법원에 제출하도록 하는 조건도 걸려 있다.</s> 전 회장은 변호인단은나카 준이치로 변호사는弘<pad>惇一郞) 변호사는 곤개인된 내용 이상의 것을 알지 못하며 현재 현재 매우 놀랐다”고</s> 정보 들어오면 법원에 제공하겠다”고 말했다.했다.</s> 교도곤 전 회장이 여권은 변호사가 보관하고 있다”고 변호인단이 여권을 주는 일은 없을 수 없다”고 말했다.했다.</s>통신은</s> 출입국재류관리청 데이터베이스를 곤 전 회장이 출국했다고 것으로 없는 것으로 확인됐다고 보도자의 설명을 토대로 전했다.</s> 관련해 교도바논 치안 당국자는 밝혔 전 회장이장으로 보이는 인

In [ ]:
# default generation
model.generate(sample['input_ids'])

tensor([[    2, 18007, 19014,  9770, 15119, 22603, 21645, 19899, 12034, 15199,
         12159, 15412, 14471, 10443, 24080, 17297, 16680, 21110, 14038,     2],
        [    2, 13328, 18067, 11466, 14499, 11786, 29195, 19966, 15569, 18751,
         11466, 14499, 11786, 29195, 19966, 16993, 14700, 14029, 16789,     2],
        [    2, 17738, 21952, 14167, 27342, 14318, 15410, 13607, 16667, 16258,
         20798, 18809, 14281, 12141, 16644, 19479, 10952, 18765, 14494,     2],
        [    2, 23436, 22915, 14086, 11207, 18497,   283,   373, 24514, 18950,
           373, 25130, 11986, 20769, 14831, 14306, 13230,  9482,   239,     2],
        [    2, 29597, 24080, 17297, 14038, 18612, 14046, 11265, 15679, 14059,
         20975,  9230, 12007, 21604, 14591, 14399, 15194, 14167, 17297,     2],
        [    2, 18007, 12034, 22246, 14311, 14196, 24175, 26787, 27834, 18335,
         22980, 14466, 14718, 15479, 14599, 14225, 14287, 16788, 23444,     2]])

In [ ]:
# beam search 결과
tokenizer.batch_decode(model.generate(sample['input_ids'], num_beams=4, max_length=512, early_stopping=False),skip_special_tokens=True, clean_up_tokenization_spaces=False)

['일본 사법당국의 출국금지 명령이 내려져 있던 카를로스 곤(65) 전 르노닛산 회장이 30일 레바논 수도 베이루트 공항에 도착했다고 미 일간 월스트리트저널(WSJ)은 보도 이후 미국의 대변인을 통해 자신이 레바논에 머물고 있다는 내용의 성명을 발표해 곤 전 회장의 변호인은 물론 수사·출입국 당국까지 당혹스럽다는 반응을 보이고 있다.',
 '트럼프의 대척점에 선 정치적 입장과 정치인으로서 신선한 이미지를 가진 엘리자베스 워런은 지난해 하반기 최고 관심 후보였다.',
 '1월 1일 새해 김정은 북한 국무위원장은 대외 메시지인 동시에 북한 주민을 향한 연설인 신년사 육성연설에서 그동안 자신이 약속했던 경제 개발과 대미 협상의 성과가 부진했던 탓이라는 분석이 나오는 가운데, 김 위원장은 나흘 짜리 전원회의를 개최해 엘리트를 단속했다.',
 '국내 이동통신 3사(KT·SK텔레콤·LG유플러스)는 산토끼(신규 고객)를 잡느라 집토끼(장기 가입자)를 위한 서비스에 인색하다는 지적을 받아왔지만 이동통신사들은 “회사 차원에서 장기 가입자는 VIP이며, 이들을 위한 혜택을 점차 늘리고 있다”고 반박하며 장기 가입자를 위한 이통 3사의 서비스를 비교해봤다.',
 '카를로스 곤(66)닛산자동차 전 회장의 ‘보석기간 중 탈출극’으로 인한 일본내 충격이 확산되고 있는 가운데 일본 언론들에 따르면 보석을 신청했던 변호사측은 “아닌 밤중에 홍두깨를 전혀 몰랐다”고 황당해 하고 있다.',
 "지난해 9월 일본을 방문한 짐 브라이든스틴 미 항공우주국(NASA) 국장이 일본 정부에 '우주 굴기'를 내세우며 세계 최초로 달의 뒷면에 탐사선을 보내는 등 빠른 속도로 우주개발을 확대하고 있는 중국을 견제하기 위해 '우주 굴기'를 내세우며 세계 최초로 달의 뒷면에 탐사선을 보내는 등 빠른 속도로 우주개발을 확대하고 있는 중국을 견제하기 위해 일본에 '유인 달 탐사 프로젝트를 함께 하자'고 정식 제안했다고 마이니치신문이 1일 보도했다."]

In [ ]:
# beam search 결과
tokenizer.batch_decode(model.generate(sample['input_ids'], num_beams=4, max_length=512, early_stopping=False),skip_special_tokens=True, clean_up_tokenization_spaces=False)

['일본 사법당국의 출국금지 명령이 내려져 있던 카를로스 곤(65) 전 르노닛산 회장이 30일 레바논 수도 베이루트 공항에 도착했다고 미 일간 월스트리트저널(WSJ)은 보도 이후 미국의 대변인을 통해 자신이 레바논에 머물고 있다는 내용의 성명을 발표해 곤 전 회장의 변호인은 물론 수사·출입국 당국까지 당혹스럽다는 반응을 보이고 있다.',
 '트럼프의 대척점에 선 정치적 입장과 정치인으로서 신선한 이미지를 가진 엘리자베스 워런은 지난해 하반기 최고 관심 후보였다.',
 '1월 1일 새해 김정은 북한 국무위원장은 대외 메시지인 동시에 북한 주민을 향한 연설인 신년사 육성연설에서 그동안 자신이 약속했던 경제 개발과 대미 협상의 성과가 부진했던 탓이라는 분석이 나오는 가운데, 김 위원장은 나흘 짜리 전원회의를 개최해 엘리트를 단속했다.',
 '국내 이동통신 3사(KT·SK텔레콤·LG유플러스)는 산토끼(신규 고객)를 잡느라 집토끼(장기 가입자)를 위한 서비스에 인색하다는 지적을 받아왔지만 이동통신사들은 “회사 차원에서 장기 가입자는 VIP이며, 이들을 위한 혜택을 점차 늘리고 있다”고 반박하며 장기 가입자를 위한 이통 3사의 서비스를 비교해봤다.',
 '카를로스 곤(66)닛산자동차 전 회장의 ‘보석기간 중 탈출극’으로 인한 일본내 충격이 확산되고 있는 가운데 일본 언론들에 따르면 보석을 신청했던 변호사측은 “아닌 밤중에 홍두깨를 전혀 몰랐다”고 황당해 하고 있다.',
 "지난해 9월 일본을 방문한 짐 브라이든스틴 미 항공우주국(NASA) 국장이 일본 정부에 '우주 굴기'를 내세우며 세계 최초로 달의 뒷면에 탐사선을 보내는 등 빠른 속도로 우주개발을 확대하고 있는 중국을 견제하기 위해 '우주 굴기'를 내세우며 세계 최초로 달의 뒷면에 탐사선을 보내는 등 빠른 속도로 우주개발을 확대하고 있는 중국을 견제하기 위해 일본에 '유인 달 탐사 프로젝트를 함께 하자'고 정식 제안했다고 마이니치신문이 1일 보도했다."]

### pipeline

In [ ]:
submission = pd.read_csv('/<your_dir>/new_sample_submission.csv')
submission

,id,summary
0,472779091,요약문
1,472779092,요약문
2,472779093,요약문
3,472779094,요약문
4,472779095,요약문
...,...,...
6591,472785682,요약문
6592,472785683,요약문
6593,472785684,요약문
6594,472785685,요약문


In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

model.to(device)

In [ ]:
submit = {"id":[],'summary':[]}
for sample in tqdm(test_dataloader):
  ids = sample['id']
  sum = tokenizer.batch_decode(model.generate(sample['input_ids'].to(device), num_beams=4, max_length=512, early_stopping=False),skip_special_tokens=True, clean_up_tokenization_spaces=False)
  submit['id'].extend(ids)
  submit['summary'].extend(sum)

  0%|          | 0/1100 [00:00<?, ?it/s]

In [ ]:
len(submit['id']),len(submit['summary'])

(6596, 6596)

In [ ]:
submission['summary'] = submit['summary']
submission

,id,summary
0,472779091,일본 사법당국의 출국금지 명령이 내려져 있던 카를로스 곤(65) 전 르노닛산 회장이...
1,472779092,트럼프의 대척점에 선 정치적 입장과 정치인으로서 신선한 이미지를 가진 엘리자베스 워...
2,472779093,1월 1일 새해 김정은 북한 국무위원장은 대외 메시지인 동시에 북한 주민을 향한 연...
3,472779094,국내 이동통신 3사(KT·SK텔레콤·LG유플러스)는 산토끼(신규 고객)를 잡느라 집...
4,472779095,카를로스 곤(66)닛산자동차 전 회장의 ‘보석기간 중 탈출극’으로 인한 일본내 충격...
...,...,...
6591,472785682,배출권시장에서 배출권거래중개회사로 거래계정을 등록하려는 자는 원활한 배출권 거래를 ...
6592,472785683,산업통상자원부와 한국가스안전공사는 불법으로 유통되는 가스 온수매트와 파티오 히터에 ...
6593,472785684,정부가 올해 안에 2050 탄소중립 시나리오와 2030년 국가 온실가스감축목표(ND...
6594,472785685,산업통상자원부는 수소차 보급을 늘리고 블루수소 생산·도입을 가속화하기 위해 수소 제...


In [ ]:
submission.to_csv("/<your_dir>/KoPEGASUS_5_46500_1_done.csv",index=False)